In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Custom libraries
from Components.TrainModel import DataModule, TEMPUS, torchscript_predict
from Components.TickerData import TickerData
from Components.BackTesting import BackTesting

# Torch ML libraries
import torch
import torch.nn as nn
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

In [ ]:
#TODO: Feature importance with SHAP values and plot
#TODO: buy signals become if prediction > current by some delta (~5%). Reverse is sell (decrease by some delta). Senstitvity analysis should be conducted to compare this delta level
#TODO: Use quantstats for a HTMl tearsheet
#TODO: Add a Echo State Networks (ESN) layer to the model
#TODO: randomly sample 50 tickers, run backtest for all of them, and plot. take average sharpe ratio, and other metrics
#TODO: PUll Russell 2000 tickers (small cap equities)
#TODO: https://www.alternativesoft.com/img/blog/updated/weighted-rank-portfolios.png

In [2]:
# Set the Wikipedia page title and section header
tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
# Clean up the dataframe
nasdaq_tickers = tickers.iloc[:, [1]].to_numpy().flatten()

In [ ]:
# Set the Wikipedia page title and section header
tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
# Clean up the dataframe
SnP_tickers = tickers.iloc[:, [0]].to_numpy().flatten()

In [ ]:
tickers = np.concatenate((nasdaq_tickers, SnP_tickers))

In [3]:
#tickers = ['IONQ','QBTS','RGTI']
training_dfs = []
stocks_dfs = []
for ticker in nasdaq_tickers:
    training_data, raw_stock_data = TickerData(ticker,years=10,prediction_window=5).process_all()
    training_dfs.append(training_data)
    stocks_dfs.append(raw_stock_data)

training_data = pd.concat(training_dfs, ignore_index=False)
stock_data = pd.concat(stocks_dfs, ignore_index=False)
stock_data

Error while processing the data for CCEP
Error while merging data for CCEP; error: "['State'] not in index"


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
Date,,,,,,,,
2015-04-06 00:00:00-04:00,75.129997,76.650002,74.750000,76.230003,2556500,0.0,0.0,ADBE
2015-04-07 00:00:00-04:00,76.070000,76.680000,75.620003,75.650002,2601300,0.0,0.0,ADBE
2015-04-08 00:00:00-04:00,75.680000,76.349998,75.120003,75.440002,1938500,0.0,0.0,ADBE
2015-04-09 00:00:00-04:00,75.620003,76.349998,75.209999,76.330002,1556200,0.0,0.0,ADBE
2015-04-10 00:00:00-04:00,76.300003,76.629997,76.000000,76.529999,1801200,0.0,0.0,ADBE
...,...,...,...,...,...,...,...,...
2025-03-24 00:00:00-04:00,208.000000,210.179993,206.600006,209.869995,1573800,0.0,0.0,ZS
2025-03-25 00:00:00-04:00,211.679993,216.389999,211.199997,215.729996,2311600,0.0,0.0,ZS
2025-03-26 00:00:00-04:00,215.250000,216.000000,209.630005,211.550003,1675700,0.0,0.0,ZS


In [1]:
import pyodbc

ImportError: dlopen(/Users/thomasaltmann/opt/anaconda3/lib/python3.8/site-packages/pyodbc.cpython-38-darwin.so, 0x0002): Library not loaded: /usr/local/opt/unixodbc/lib/libodbc.2.dylib
  Referenced from: <157C2418-FBB4-3F9F-B952-97899DBF4398> /Users/thomasaltmann/opt/anaconda3/lib/python3.8/site-packages/pyodbc.cpython-38-darwin.so
  Reason: tried: '/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/usr/local/lib/libodbc.2.dylib' (no such file), '/usr/lib/libodbc.2.dylib' (no such file, not in dyld cache)

In [4]:
# %%
# Import your stock_data dataframe into Azure SQL using SQLAlchemy

import os
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.engine import URL
import urllib.parse


def upload_stock_data_using_sqlalchemy():
    try:
        # Get connection string from environment variables
        #connection_string = os.environ["AZURE_SQL_CONNECTIONSTRING"]
        connection_string = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:projectdeepgreen.database.windows.net,1433;Database=us_equities;Uid=taltmann;Pwd= Millie5367!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

        # Create SQLAlchemy engine
        # Note: For Azure SQL with token authentication, we'll need to use a different approach
        # than a standard connection string

        # Option 1: If your connection string is ODBC format:
        engine = create_engine(f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}")

        # Option 2: If you need to use token authentication:
        # This would require additional setup similar to your pyodbc approach

        # Reset index to make Date a column instead of the index
        data_to_upload = stock_data.reset_index().rename(columns={'index': 'Date'})

        # Drop columns not in the table schema (Dividends and Stock Splits)
        data_to_upload = data_to_upload[['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']]

        # Upload the dataframe to SQL
        # The to_sql method will handle the insert statements for you
        data_to_upload.to_sql(
            name='nasdaq_1day',
            con=engine,
            if_exists='append',  # 'replace' if you want to overwrite, 'append' to add to existing
            index=False,
            chunksize=1000  # Process in batches of 1000 rows
        )

        print(f"Successfully uploaded {len(data_to_upload)} records to nasdaq_1day table")

    except Exception as e:
        print(f"Error uploading data: {str(e)}")


# Execute the function
upload_stock_data_using_sqlalchemy()


Error uploading data: dlopen(/Users/thomasaltmann/opt/anaconda3/lib/python3.8/site-packages/pyodbc.cpython-38-darwin.so, 0x0002): Library not loaded: /usr/local/opt/unixodbc/lib/libodbc.2.dylib
  Referenced from: <157C2418-FBB4-3F9F-B952-97899DBF4398> /Users/thomasaltmann/opt/anaconda3/lib/python3.8/site-packages/pyodbc.cpython-38-darwin.so
  Reason: tried: '/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/usr/local/lib/libodbc.2.dylib' (no such file), '/usr/lib/libodbc.2.dylib' (no such file, not in dyld cache)


In [2]:
import os
import pyodbc, struct
from azure import identity

connection_string = os.environ["AZURE_SQL_CONNECTIONSTRING"]



def get_conn():
    credential = identity.DefaultAzureCredential(exclude_interactive_browser_credential=False)
    token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by microsoft in msodbcsql.h
    conn = pyodbc.connect(connection_string)
    return conn

# %%
# Import your stock_data dataframe into Azure SQL

def upload_stock_data_to_azure():
    # Reset index to make Date a column instead of the index
    data_to_upload = stock_data.reset_index().rename(columns={'index': 'Date'})

    # Drop columns not in the table schema (Dividends and Stock Splits)
    data_to_upload = data_to_upload[['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']]

    # Connect to Azure SQL
    try:
        with get_conn() as conn:
            cursor = conn.cursor()

            # Use fast_executemany for better performance with large datasets
            cursor.fast_executemany = True

            # Prepare the SQL INSERT statement
            insert_sql = """
            INSERT INTO nasdaq_1day (Date, Ticker, [Open], High, Low, [Close], Volume)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """

            # Convert dataframe to list of tuples
            records = list(data_to_upload.itertuples(index=False, name=None))

            # Execute the insert
            cursor.executemany(insert_sql, records)
            conn.commit()

            print(f"Successfully uploaded {len(data_to_upload)} records to nasdaq_1day table")

    except Exception as e:
        print(f"Error uploading data: {str(e)}")


# Execute the function
upload_stock_data_to_azure()


ImportError: dlopen(/Users/thomasaltmann/opt/anaconda3/lib/python3.8/site-packages/pyodbc.cpython-38-darwin.so, 0x0002): Library not loaded: /usr/local/opt/unixodbc/lib/libodbc.2.dylib
  Referenced from: <E38C7B25-7727-3975-BBA6-24065FE028D6> /Users/thomasaltmann/opt/anaconda3/lib/python3.8/site-packages/pyodbc.cpython-38-darwin.so
  Reason: tried: '/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/usr/local/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/usr/local/lib/libodbc.2.dylib' (no such file), '/usr/lib/libodbc.2.dylib' (no such file, not in dyld cache)

In [ ]:
#Best config: {'lr': 4.390449033248878e-05, 'hidden_size': 256, 'num_layers': 1, 'dropout': 0.3477694988633191, 'weight_decay': 0.0001801390872725824, 'batch_size': 16, 'window_size': 10, 'grad_clip_norm': 0.8393802881451728}

config = {
    "lr": 4.390449033248878e-05,
    "weight_decay": 0.0001801390872725824,
    "hidden_size": 256,
    "num_layers": 1,
    "dropout": 0.3477694988633191,
    "batch_size": 16,
    "window_size": 50,
    "clip_size": 0.8393802881451728,
    "epochs": 20,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

data_module = DataModule(training_data, window_size=config["window_size"], batch_size=config["batch_size"])
config["input_size"] = data_module.num_features

# Instantiate the model
model = TEMPUS(config,scaler=data_module.scaler)
# Set up loss and optimizer
criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
# Train Model
history = model.train_model(data_module.train_loader, data_module.test_loader, criterion, optimizer, config["epochs"])

In [ ]:
training_fig = model.plot_training_history()
training_fig.show()

In [ ]:
# Export the trained TEMPUS model
script_path = model.export_model_to_torchscript(
    save_path="Models/Tempus_v2.pt",
    data_loader=data_module.test_loader,
    device="cpu"
)

In [ ]:
ticker = "PLTR"  # Replace with your ticker of interest
out_of_sample_data, raw_stock_data = TickerData(ticker, years=1, prediction_window=5).process_all()

# Load the model and make predictions
preds_df = torchscript_predict(
    model_path="Models/Tempus_v2.pt",
    input_df=out_of_sample_data,
    device="cpu",
    window_size=50,
    target_col="shifted_prices"
)

preds_df = pd.merge(preds_df, raw_stock_data[['Open', 'High', 'Low', 'Volume','Close']], left_index=True, right_index=True, how='left')

fig = go.Figure()
fig.add_trace(go.Scatter(y=preds_df['Predicted'], x=preds_df.index, mode='lines', name='Predicted',line=dict(color="Grey")))
fig.add_trace(go.Scatter(y=preds_df['Close'], x=preds_df.index, mode='lines', name='Close (Unshifted)',line=dict(color="Blue")))
fig.add_trace(go.Scatter(y=preds_df['Actual'], x=preds_df.index, mode='lines', name='Close (Shifted)'))
fig.update_layout(title=f'Prediction for {ticker}', xaxis_title='Date', yaxis_title='Price (USD)',height=600,legend=dict(orientation="h", yanchor="bottom", y=1.02))
#fig.show()

In [ ]:
from Components.BackTesting import BackTesting
import pandas as pd

initial_capital = 1000.0
ticker = 'PLTR'
backtester = BackTesting(preds_df, ticker, initial_capital,pct_change_entry=0.05,pct_change_exit=0.02)
results, _ = backtester.run_simulation()
trades_fig, value_fig, exposure_fig = backtester.plot_performance()
#trades_fig.show()
#value_fig.show()
#exposure_fig.show()

In [ ]:
import quantstats as qs

returns = backtester.pf.returns()

#html = qs.reports.full(returns, "NDAQ")
pd.DataFrame(qs.reports.metrics(returns, "NDAQ",mode='full',rf=0.0025, display=False))


In [ ]:
class TCNBlock(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, dilation, padding, dropout=0.2):
        super(TCNBlock, self).__init__()

        self.conv1 = nn.Conv1d(
            in_channels=input_dim,
            out_channels=output_dim,
            kernel_size=kernel_size,
            dilation=dilation,
            padding=padding
        )
        self.norm1 = nn.BatchNorm1d(output_dim)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(
            in_channels=output_dim,
            out_channels=output_dim,
            kernel_size=kernel_size,
            dilation=dilation,
            padding=padding
        )
        self.norm2 = nn.BatchNorm1d(output_dim)
        self.relu2 = nn.ReLU()  # Added missing relu2 activation
        self.dropout2 = nn.Dropout(dropout)

        # Residual connection if dimensions don't match
        self.residual = nn.Conv1d(input_dim, output_dim, 1) if input_dim != output_dim else nn.Identity()

    def forward(self, x):

        # First conv block
        # Residual input
        residual = self.residual(x)

        # First conv block
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        # Second conv block
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.relu2(out)
        out = self.relu2(out)  # Correctly use relu2
        out = self.dropout2(out)

        # Return to original shape
        # Add the residual and pass through final activation
        return self.relu1(out + residual)  # Fixed to use relu1 for the final activation

In [ ]:
class EchoStateNetwork(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, spectral_radius=0.9,
                 sparsity=0.1, noise=0.001, bidirectional=False):
        super(EchoStateNetwork, self).__init__()

        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.bidirectional = bidirectional

        # Input weights (fixed)
        self.register_buffer('W_in', self._initialize_input_weights())

        # Reservoir weights (fixed)
        self.register_buffer('W', self._initialize_reservoir_weights())

        # Output weights (trainable)
        self.W_out = nn.Linear(reservoir_size, output_size)

        if bidirectional:
            # Second set of weights for backward direction
            self.register_buffer('W_in_reverse', self._initialize_input_weights())
            self.register_buffer('W_reverse', self._initialize_reservoir_weights())
            self.W_out_reverse = nn.Linear(reservoir_size, output_size)
            # Combined output
            self.W_combined = nn.Linear(output_size * 2, output_size)

    def _initialize_input_weights(self):
        W_in = torch.zeros(self.reservoir_size, self.input_size)
        W_in = torch.nn.init.xavier_uniform_(W_in)
        return W_in

    def _initialize_reservoir_weights(self):
        # Create sparse matrix
        W = torch.zeros(self.reservoir_size, self.reservoir_size)
        num_connections = int(self.sparsity * self.reservoir_size * self.reservoir_size)
        indices = torch.randperm(self.reservoir_size * self.reservoir_size)[:num_connections]
        rows = indices // self.reservoir_size
        cols = indices % self.reservoir_size
        values = torch.randn(num_connections)
        W[rows, cols] = values

        # Scale to desired spectral radius
        eigenvalues = torch.linalg.eigvals(W)
        max_eigenvalue = torch.max(torch.abs(eigenvalues))
        W = W * (self.spectral_radius / max_eigenvalue)
        return W

    def _reservoir_step(self, x, h_prev, W_in, W):
        """Execute one step of the reservoir"""
        # h_new = tanh(W_in @ x + W @ h_prev + noise)
        h_new = torch.tanh(torch.mm(x, W_in.t()) + torch.mm(h_prev, W.t()) +
                           self.noise * torch.randn(h_prev.shape, device=h_prev.device))
        return h_new

    def forward(self, x):
        """
        x: input tensor of shape (batch_size, seq_len, input_size)
        """
        batch_size, seq_len, _ = x.size()

        # Forward pass
        h = torch.zeros(batch_size, self.reservoir_size, device=x.device)
        outputs_forward = []

        for t in range(seq_len):
            h = self._reservoir_step(x[:, t], h, self.W_in, self.W)
            outputs_forward.append(self.W_out(h))

        outputs_forward = torch.stack(outputs_forward, dim=1)  # (batch_size, seq_len, output_size)

        if not self.bidirectional:
            return outputs_forward

        # Backward pass for bidirectional ESN
        h_reverse = torch.zeros(batch_size, self.reservoir_size, device=x.device)
        outputs_reverse = []

        for t in range(seq_len - 1, -1, -1):
            h_reverse = self._reservoir_step(x[:, t], h_reverse, self.W_in_reverse, self.W_reverse)
            outputs_reverse.insert(0, self.W_out_reverse(h_reverse))

        outputs_reverse = torch.stack(outputs_reverse, dim=1)  # (batch_size, seq_len, output_size)

        # Combine forward and backward outputs
        combined = torch.cat((outputs_forward, outputs_reverse), dim=2)
        return self.W_combined(combined)

In [ ]:
#ticker.get_balance_sheet(freq='quarterly')
#ticker.get_calendar()
#ticker.get_cash_flow(freq='quarterly')
#earnings_data = ticker.get_earnings_dates()
#income_statement = ticker.get_income_stmt(freq='yearly').T
#ticker.get_institutional_holders()
#ticker.get_recommendations()
#ticker.get_sustainability()

In [ ]:
# define a function to fetch the options data for a given ticker symbol
#def fetch_options_data(ticker_symbol):
    #ticker = yf.Ticker(ticker_symbol)
#    options_dates = ticker.options
#    options_data = ticker.option_chain(date='2025-03-21')
#    return options_data.calls, options_data.puts
##ionq_stock_data = ionq_stock_data.sort_values(by='Date', ascending=False)